# Deploying ADK Agent on Vertex AI Agent Engine

## Overview

This notebook guides you through deploying and managing a conversational AI agent to **Vertex AI Agent Engine**. 

## Learning Goals

By the end of this notebook, you will understand how to:
* Set up required configuration and deploy ADK **Agents** to **Vertex AI Agent Engine**.
* Use the ADK **RemoteAgent** ans **RemoteSession** to execute agent interactions.
* Use the ADK to Manage (list/update) and delete agents that you have deployed to Vertex AI Agent Engine.

## Deployment Options
Your ADK agent can be deployed to a range of different environments based on your needs for production readiness or custom flexibility:

#### Agent Engine in Vertex AI
Agent Engine is a fully managed auto-scaling service on Google Cloud specifically designed for deploying, managing, and scaling AI agents built with frameworks such as ADK.

Learn more about deploying your agent to Vertex AI Agent Engine.

#### Cloud Run
Cloud Run is a managed auto-scaling compute platform on Google Cloud that enables you to run your agent as a container-based application.

Learn more about deploying your agent to Cloud Run.

#### Google Kubernetes Engine (GKE)
Google Kubernetes Engine (GKE) is a managed Kubernetes service of Google Cloud that allows you to run your agent in a containerized environment. GKE is a good option if you need more control over the deployment as well as for running Open Models.

Learn more about deploying your agent to GKE.

## Setup
This lab needs a special kernel to run, please run the following cell.
**NOTE: You can skip this step if you have already built the ADK Kernel from the previous Lab**

In [1]:
# !echo "Kernel installation started."
# !cd ../../.. && make adk_kernel > /dev/null 2>&1
# !echo "Kernel installation completed."

When it's completed, select the **`ADK Kernel`** on the top right before going forward in the notebook.<br>
It may take ~1 minutes until the kernel is shown after the installation.

## Install Packages

In [2]:
import asyncio
import importlib
import json
import os
import warnings

import pandas as pd
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm  # For multi-model support
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.genai import types  # For creating message Content/Parts
from IPython.display import HTML, Markdown, display

# Ignore all warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)

In [3]:
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"  # Use Vertex AI API

In [4]:
%%bash
echo > adk_agents/.env "GOOGLE_CLOUD_LOCATION=$GOOGLE_CLOUD_LOCATION
GOOGLE_GENAI_USE_VERTEXAI=$GOOGLE_GENAI_USE_VERTEXAI
"

In [5]:
MODEL = "gemini-2.0-flash"

## Basic App: Weather Lookup
**NOTE: You can skip this step if you have already created "./adk_agents/agent1_weather_lookup/tools.py" file from the previous Lab**

Let's begin by building the fundamental component of our Weather Bot: a single agent capable of performing a specific task – looking up weather information. This involves creating two core pieces:

- A Tool: A Python function that equips the agent with the ability to fetch weather data.
- An Agent: The AI "brain" that understands the user's request, knows it has a weather tool, and decides when and how to use it.

### Define the Tool (get_weather)

In ADK, **Tools** are the building blocks that give agents concrete capabilities beyond just text generation. They are typically regular Python functions that perform specific actions, like calling an API, querying a database, or performing calculations.

Our first tool will provide a *mock* weather report. This allows us to focus on the agent structure without needing external API keys yet. Later, you could easily swap this mock function with one that calls a real weather service.

**Key Concept: Docstrings are Crucial\!** The agent's LLM relies heavily on the function's **docstring** to understand:

* *What* the tool does.  
* *When* to use it.  
* *What arguments* it requires (`city: str`).  
* *What information* it returns.

**Best Practice:** Write clear, descriptive, and accurate docstrings for your tools. This is essential for the LLM to use the tool correctly.

In [6]:
%%writefile ./adk_agents/agent1_weather_lookup/tools.py
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

Overwriting ./adk_agents/agent1_weather_lookup/tools.py


### Define the Agent (`weather_agent`)

### NOTE: You can skip this step if you have already created "./adk_agents/agent1_weather_lookup/agent.py" file from the previous Lab

Now, let's create the **Agent** itself. An `Agent` in ADK orchestrates the interaction between the user, the LLM, and the available tools.

We configure it with several key parameters:

* `name`: A unique identifier for this agent (e.g., "weather\_agent\_v1").  
* `model`: Specifies which LLM to use (e.g., `gemini-2.0-flash`).
* `description`: A concise summary of the agent's overall purpose. This becomes crucial later when other agents need to decide whether to delegate tasks to *this* agent.  
* `instruction`: Detailed guidance for the LLM on how to behave, its persona, its goals, and specifically *how and when* to utilize its assigned `tools`.  
* `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[get_weather]`).

**Best Practices:** 
- Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).
- Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.

In [7]:
%%writefile ./adk_agents/agent1_weather_lookup/agent.py
from google.adk.agents import Agent
MODEL = "gemini-2.0-flash"

from .tools import get_weather

root_agent = Agent(
    name="weather_agent_v1",
    model=MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

Overwriting ./adk_agents/agent1_weather_lookup/agent.py


In [8]:
from adk_agents.agent1_weather_lookup import agent

importlib.reload(agent)  # Force reload

# Example tool usage (optional test)
print(agent.get_weather("New York"))
print(agent.get_weather("Dublin"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Dublin ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Dublin'."}


In [9]:
APP_NAME = "weather_info_app"
USER_ID = "user_1"
SESSION_ID = "session_001"  # Using a fixed ID for simplicity
EXPERIMENT_NAME = "weather-agent-v1"

# Deploy to Vertex AI Agent Engine
Agent Engine is a fully managed Google Cloud service enabling developers to deploy, manage, and scale AI agents in production. 
Agent Engine handles the infrastructure to scale agents in production so you can focus on creating intelligent and impactful applications.

In [10]:
LOCATION = "us-central1"
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET_NAME = f"agent-evaluation-{PROJECT}-bucket"
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Checking for the existence of BUCKET. Creating it if it doesn't exist:**

In [11]:
!gsutil ls $BUCKET_URI || gsutil mb -l $LOCATION $BUCKET_URI

gs://agent-evaluation-oleksandr-demo-bucket/agent_engine/
gs://agent-evaluation-oleksandr-demo-bucket/byod-eval/
gs://agent-evaluation-oleksandr-demo-bucket/multiple-metric-eval/
gs://agent-evaluation-oleksandr-demo-bucket/reasoning-metric-eval/
gs://agent-evaluation-oleksandr-demo-bucket/response-metric-eval/
gs://agent-evaluation-oleksandr-demo-bucket/single-metric-eval/


### Initialization

In [12]:
import os

import vertexai
from absl import app, flags
from dotenv import load_dotenv
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

vertexai.init(
    project=PROJECT,
    location=LOCATION,
    staging_bucket=BUCKET_URI
)

### Prepare your agent for Agent Engine¶
Use reasoning_engines.AdkApp() to wrap your agent to make it deployable to Agent Engine

In [13]:
adk_app = AdkApp(agent=agent.root_agent, enable_tracing=True)

### Deploy your agent to Agent Engine

Now you're ready to deploy your agent to Agent Engine in Vertex AI by calling `agent_engines.create()` along with:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

This step may take several minutes to finish.

In [14]:
DISPLAY_NAME = agent.root_agent.name

remote_app = agent_engines.create(
    adk_app,
    display_name=DISPLAY_NAME,
    description="Weather Agent v1",
    requirements=[
        "google-adk (>=1.5.0)",
        "google-genai (>=1.5.0,<2.0.0)",
    ],
    extra_packages=["./adk_agents/"],
)

print(f"Created remote agent: {remote_app.resource_name}")


Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.101.0', 'pydantic': '2.11.7'}


INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.101.0', 'pydantic': '2.11.7'}


The following requirements are missing: {'cloudpickle', 'google-cloud-aiplatform', 'pydantic'}


The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}


INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}


...

To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888')


Created remote agent: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888


### Try ADK agent on Agent Engine
#### Create session (remote)

In [15]:
remote_session = remote_app.create_session(user_id=USER_ID)

#### Expected output for create_session (remote):

In [16]:
remote_session

{'appName': '1576356626602917888',
 'events': [],
 'state': {},
 'id': '6141980254652071936',
 'userId': 'user_1',
 'lastUpdateTime': 1753200758.889636}

#### id is the session ID, and app_name is the resource ID of the deployed agent on Agent Engine.

#### List sessions (remote)

In [17]:
remote_app.list_sessions(user_id=USER_ID)

{'sessions': [{'appName': '1576356626602917888',
   'events': [],
   'state': {},
   'id': '6141980254652071936',
   'userId': 'user_1',
   'lastUpdateTime': 1753200758.889636}]}

#### Get a specific session (remote)
While using your agent locally, session ID is stored in session.id, when using your agent remotely on Agent Engine, session ID is stored in remote_session["id"].

In [18]:
remote_app.get_session(user_id=USER_ID, session_id=remote_session["id"])

{'appName': '1576356626602917888',
 'events': [],
 'state': {},
 'id': '6141980254652071936',
 'userId': 'user_1',
 'lastUpdateTime': 1753200758.889636}

#### Send queries to your agent (remote)

Now you can send a prompt to your remote agent using .stream_query to test that it's working as expected:

In [19]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="What's the weather in New York?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-e6b8fec2-c254-468e-a2d2-2d3efa163419', 'args': {'city': 'New York'}, 'name': 'get_weather'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 6, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 6}], 'prompt_token_count': 209, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 209}], 'total_token_count': 215, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-e8a4d7e8-5e22-4c61-8bed-d236a6e31dd2', 'author': 'weather_agent_v1', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '451e9bdf-8c53-4156-8aa2-0e35d4d6687e', 'timestamp': 1753200761.572891}
{'content': {'parts': [{'function_response': {'id': 'adk-e6b8fec2-c254-468e-a2d2-2d3efa163419', 'name': 'get_weather', 'response': {'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}}}], 'role': 'user'}, 'invocation_id': 'e-e8a4d7e8-5e22-4

#### Format the stream output as a key-value pairs:

In [20]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="whats the weather in Dublin",
):
    for key, label in event.items():
        print("\n------>")
        print(f"{key} : {label}")


------>
content : {'parts': [{'function_call': {'id': 'adk-14442f54-da19-4294-8493-cc862920e6e5', 'args': {'city': 'Dublin'}, 'name': 'get_weather'}}], 'role': 'model'}

------>
usage_metadata : {'candidates_token_count': 5, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 5}], 'prompt_token_count': 261, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 261}], 'total_token_count': 266, 'traffic_type': 'ON_DEMAND'}

------>
invocation_id : e-2f7ef25b-76f9-406f-84fd-56d7b1acfc4a

------>
author : weather_agent_v1

------>
actions : {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}

------>
long_running_tool_ids : []

------>
id : cbd4314b-7dbc-4884-a065-c5a2c7edfcda

------>
timestamp : 1753200764.170943

------>
content : {'parts': [{'function_response': {'id': 'adk-14442f54-da19-4294-8493-cc862920e6e5', 'name': 'get_weather', 'response': {'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Dublin'."}}}], 

### Manage deployed agents
Deployed agents are resources of type reasoningEngine in Vertex AI.

#### List all deployed agents for a given project and location:
Each deployed agent has a unique identifier ***Name*** and fully qualified resource name ***Resource Name***

In [21]:
agents_list = agent_engines.list()
for remote_agent in agents_list:
    print("\n------")
    print(f"Display Name: {remote_agent.display_name}")
    print(f"Name: {remote_agent.name}")
    print(f"Resource Name: {remote_agent.resource_name}")
    print(f"Create Time: {remote_agent.create_time}")


------
Display Name: weather_agent_v1
Name: 1576356626602917888
Resource Name: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888
Create Time: 2025-07-22 16:09:39.060096+00:00


#### Filter results by display_name

In [22]:
DISPLAY_NAME = "weather_agent_v1"

In [23]:
AGENT_RESOURCE_NAME="projects/ ... /locations/us-central1/reasoningEngines/ ... "
AGENT_RESOURCE_ID=" ... "
agents_list = agent_engines.list(filter=f'display_name="{DISPLAY_NAME}"')
if agents_list:
    AGENT_RESOURCE_ID = remote_agent.name
    print(f"Agent Name: {remote_agent.name}")
    AGENT_RESOURCE_NAME = remote_agent.resource_name
    print(f"Resource Name: {remote_agent.resource_name}")

Agent Name: 1576356626602917888
Resource Name: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888


#### Get a deployed agent
The following code lets you get a specific deployed agent:

In [24]:
from vertexai import agent_engines

remote_agent = agent_engines.get(AGENT_RESOURCE_NAME)
remote_agent

resource name: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888

Alternately, you can provide the fully qualified resource name:

In [25]:
from vertexai import agent_engines

remote_agent = agent_engines.get(AGENT_RESOURCE_ID)
remote_agent

resource name: projects/oleksandr-demo/locations/us-central1/reasoningEngines/1576356626602917888

### Update a deployed agent
You can update one or more fields of the deployed agent at the same time, but you have to specify at least one of the fields to be updated. The amount of time it takes to update the deployed agent depends on the update being performed, but it generally takes between a few seconds to a few minutes.

Lets change out tool source code by adding one more city to the weather data mock:

In [26]:
!cp ./adk_agents/agent1_weather_lookup/tools.py ./adk_agents/agent2_sub_agent/

In [27]:
%%writefile -a ./adk_agents/agent2_sub_agent/tools.py

def say_hello(name: str = "there") -> str:
    """Provides a simple greeting, optionally addressing the user by name.

    Args:
        name (str, optional): The name of the person to greet. Defaults to "there".

    Returns:
        str: A friendly greeting message.
    """
    if name is None or name.strip() == "":
        name = "there"
    print(f"--- Tool: say_hello called with name: {name} ---")
    return f"Hello, {name}!"

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

Appending to ./adk_agents/agent2_sub_agent/tools.py


In [28]:
from adk_agents.agent2_sub_agent import tools

print(tools.say_hello("Alice"))
print(tools.say_goodbye())

--- Tool: say_hello called with name: Alice ---
Hello, Alice!
--- Tool: say_goodbye called ---
Goodbye! Have a great day.


In [29]:
%%writefile ./adk_agents/agent2_sub_agent/agent.py
from google.adk.agents import Agent
MODEL = "gemini-2.0-flash"

from .tools import get_weather, say_hello, say_goodbye

# --- Greeting Agent ---
greeting_agent = Agent(
    model=MODEL,
    name="greeting_agent",
    instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                "Use the 'say_hello' tool to generate the greeting. "
                "If the user provides their name, make sure to pass it to the tool. "
                "Do not engage in any other conversation or tasks.",
    description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
    tools=[say_hello],
)

Overwriting ./adk_agents/agent2_sub_agent/agent.py


In [30]:
%%writefile -a ./adk_agents/agent2_sub_agent/agent.py

# --- Farewell Agent ---
farewell_agent = Agent(
    model=MODEL,
    name="farewell_agent",
    instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                "Do not perform any other actions.",
    description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
    tools=[say_goodbye],
)

Appending to ./adk_agents/agent2_sub_agent/agent.py


In [31]:
%%writefile -a ./adk_agents/agent2_sub_agent/agent.py

root_agent = Agent(
    name="weather_agent_v2", # Give it a new version name
    model=MODEL,
    description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
    instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                "You have specialized sub-agents: "
                "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                "If it's a weather request, handle it yourself using 'get_weather'. "
                "For anything else, respond appropriately or state you cannot handle it.",
    tools=[get_weather],
    sub_agents=[greeting_agent, farewell_agent]
)

Appending to ./adk_agents/agent2_sub_agent/agent.py


In [32]:
from adk_agents.agent2_sub_agent import agent as agent2

importlib.reload(agent2)  # Force reload

<module 'adk_agents.agent2_sub_agent.agent' from '/home/jupyter/adk_deployment/asl-ml-immersion/notebooks/vertex_genai/solutions/adk_agents/agent2_sub_agent/agent.py'>

Test for local changes:

In [33]:
# Example tool usage (optional test)
print(tools.say_hello("Alice"))
print(agent2.get_weather("New York"))
print(tools.say_goodbye())

--- Tool: say_hello called with name: Alice ---
Hello, Alice!
--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: say_goodbye called ---
Goodbye! Have a great day.


#### To update a deployed agent (corresponding to AGENT_RESOURCE_ID) to an updated agent (corresponding to UPDATED_AGENT_APP):

In [34]:
from vertexai import agent_engines

UPDATED_AGENT_APP = AdkApp(agent=agent2.root_agent, enable_tracing=True)

agent_engines.update(
    resource_name=AGENT_RESOURCE_ID,
    agent_engine=UPDATED_AGENT_APP,
    description="Weather Agent v2",
    display_name=agent2.root_agent.name,
    requirements=[
        "google-adk (>=1.5.0)",
        "google-genai (>=1.5.0,<2.0.0)",
    ],
    extra_packages=["./adk_agents/"],
)

Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.101.0', 'pydantic': '2.11.7'}


INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.101.0', 'pydantic': '2.11.7'}


The following requirements are missing: {'cloudpickle', 'google-cloud-aiplatform', 'pydantic'}


The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}


INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}


...

Update Agent Engine backing LRO: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888/operations/8060847088453287936


INFO:vertexai.agent_engines:Update Agent Engine backing LRO: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888/operations/8060847088453287936


Agent Engine updated. Resource name: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888


INFO:vertexai.agent_engines:Agent Engine updated. Resource name: projects/167415611431/locations/us-central1/reasoningEngines/1576356626602917888


resource name: projects/oleksandr-demo/locations/us-central1/reasoningEngines/1576356626602917888

Checking changed result, after agent update:

In [35]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="Bye!",
):
    for key, label in event.items():
        print("\n------>")
        print(f"{key} : {label}")


------>
content : {'parts': [{'function_call': {'id': 'adk-4b20e249-f030-4e09-9985-658366cc8862', 'args': {'agent_name': 'farewell_agent'}, 'name': 'transfer_to_agent'}}], 'role': 'model'}

------>
usage_metadata : {'candidates_token_count': 12, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 12}], 'prompt_token_count': 751, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 751}], 'total_token_count': 763, 'traffic_type': 'ON_DEMAND'}

------>
invocation_id : e-dac8575a-87be-473c-8292-3c7f34cd9e8e

------>
author : weather_agent_v2

------>
actions : {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}

------>
long_running_tool_ids : []

------>
id : e7cd861c-5fc8-46b3-a77b-928beff1c42b

------>
timestamp : 1753200945.844265

------>
content : {'parts': [{'function_response': {'id': 'adk-4b20e249-f030-4e09-9985-658366cc8862', 'name': 'transfer_to_agent', 'response': {'result': None}}}], 'role': 'user'}

------>
invocation_id : e-dac857

### Clean up
After you have finished, it is a good practice to clean up your cloud resources. 
You can delete the deployed Agent Engine instance by using the next code:

In [36]:
# remote_app.delete(force=True)

Alternatively, you can call agent_engines.delete() to delete the deployed agent corresponding to AGENT_RESOURCE_ID in the following way:


In [37]:
#agent_engines.delete(AGENT_RESOURCE_ID)

Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.